In [2]:
import os
import time
import json
import tushare as ts

In [3]:
ts.set_token('aecca28adc0d5a7764b748ccd48bef923d81314ae47b4d44d51fce67')
tsp = ts.pro_api()

In [7]:
class Utils:
    
    SYMBOLS_FILE = os.path.join(os.getcwd(), "symbols.json")
    SUSPENDED_SYMBOLS_FILE = os.path.join(os.getcwd(), "suspended_symbols.json")
    
    @staticmethod
    def get_symbols():
        if not os.path.exists(Utils.SYMBOLS_FILE):
            return []

        with open(Utils.SYMBOLS_FILE) as f:
            return json.load(f)
        
    @staticmethod
    def get_suspended_symbols():
        if not os.path.exists(Utils.SUSPENDED_SYMBOLS_FILE):
            return []

        with open(Utils.SUSPENDED_SYMBOLS_FILE) as f:
            return json.load(f)
        
    @staticmethod
    def get_running_symbols():
        return list([symbol for symbol in Utils.get_symbols() if symbol not in Utils.get_suspended_symbols()])
        
    @staticmethod
    def update_symbols():
        df = tsp.stock_basic(fields='symbol,name,market,area,industry,list_date')
        df = df.loc[df['list_date']<=time.strftime('%Y%m%d', time.localtime())]
        symbols = df['symbol'].to_list()
        with open(Utils.SYMBOLS_FILE, "w") as f:
            f.write(json.dumps(symbols))
            
        today = time.strftime('%Y%m%d')
        suspended_symbols = list([x[:6] for x in tsp.suspend_d(suspend_type='S', trade_date=today)['ts_code'].to_list()])
        with open(Utils.SUSPENDED_SYMBOLS_FILE, "w") as f:
            f.write(json.dumps(suspended_symbols))

    @staticmethod
    def is_closed_day(day=None):
        if day is None:
            day = time.strftime('%Y%m%d')
        df = tsp.trade_cal(exchange='SSE', start_date=day, end_date=day)
        if df.iloc[0]['is_open'] == 1:
            return False
        else:
            return True
        
    @staticmethod
    def get_pretrade_date(day=None):
        if day is None:
            day = time.strftime('%Y%m%d')
        df = tsp.trade_cal(exchange='SSE', start_date=day, end_date=day, fields="exchange,cal_date,is_open,pretrade_date")
        return df.at[0,'pretrade_date']
    
    @staticmethod
    def get_last_trade_date():
        day = time.strftime('%Y%m%d')
        df = tsp.trade_cal(exchange='SSE', start_date=day, end_date=day, fields="exchange,cal_date,is_open,pretrade_date")
        if df.iloc[0]['is_open'] == 1:
            return day
        else:
            return df.at[0,'pretrade_date']
        
    @staticmethod
    def get_check_points(interval=5):
        
        trading_times = [
            '09:14:50',
            ['09:15:00', '09:25:00'],
            '09:25:15',
            ['09:30:00', '11:30:00'],
            '11:30:10',
            ['13:00:00', '15:00:00'],
            '15:00:10'
        ]
        
        check_points = []
        today = time.strftime('%Y-%m-%d')
        for time_range in trading_times:
            if type(time_range) == list:
                start = int(time.mktime(time.strptime(f'{today} {time_range[0]}', '%Y-%m-%d %H:%M:%S')))
                end = int(time.mktime(time.strptime(f'{today} {time_range[1]}', '%Y-%m-%d %H:%M:%S')))
                check_points.append(list([ _ for _ in range(start, end+interval, interval) ]))
            else:
                check_points.append([int(time.mktime(time.strptime(f'{today} {time_range}', '%Y-%m-%d %H:%M:%S')))])
        
        return sum(check_points, [])

In [7]:
Utils.update_symbols()

In [8]:
symbols = list([symbol for symbol in Utils.get_symbols() if symbol not in Utils.get_suspended_symbols()])
len(symbols)

4126

In [34]:
# today = time.strftime('%Y%m%d')
Utils.is_closed_day()

False

In [35]:
Utils.get_pretrade_date()

'20210115'

In [36]:
Utils.get_last_trade_date()

'20210118'

In [8]:
suspended = Utils.get_suspended_symbols()

In [9]:
'600634' in suspended

False

In [12]:
from datetime import datetime

In [16]:
%%timeit
datetime.strptime('2021-01-14 09:25:00', '%Y-%m-%d %H:%M:%S').timestamp()

10.1 µs ± 29.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [16]:
today = time.strftime('%Y%m%d')
today

'20210119'

In [20]:
'600634' in tsp.suspend_d(suspend_type='S', trade_date='20201208')['ts_code']

False

In [14]:
tsp.suspend(ts_code='600634.SH')

,ts_code,suspend_date,resume_date,suspend_reason
0,600634.SH,20201208,None,重要公告
1,600634.SH,20201207,None,重要公告
2,600634.SH,20201204,None,重要公告
3,600634.SH,20201203,None,重要公告
4,600634.SH,20201202,None,重要公告
...,...,...,...,...
820,600634.SH,20010525,None,关联交易
821,600634.SH,20010416,None,公布2000年年报
822,600634.SH,20001227,None,选举管理层
823,600634.SH,20001226,None,召开股东大会


In [15]:
tsp.query('suspend', suspend_date=today)

,ts_code,suspend_date,resume_date,suspend_reason
0,600723.SH,20210119,None,筹划重大资产重组
1,600859.SH,20210119,None,筹划重大资产重组
2,605005.SH,20210119,None,股票价格异常波动
3,000528.SZ,20210119,None,重大事项
4,600086.SH,20210119,None,刊登重要公告
5,601866.SH,20210119,None,重大事项
6,000885.SZ,20210119,None,筹划重大资产重组
7,000826.SZ,20210119,None,筹划重大资产重组
8,601880.SH,20210119,20210122,重要公告
9,600317.SH,20210119,None,重要公告
